In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv('day6_data1.csv')
data

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [2]:
X=data[['alcohol','sugar','pH']].to_numpy() #alcohol	sugar	pH 분할 속성
Y=data['class'].to_numpy()

In [3]:
#학습을 할때 사후가지치기를 할때 무조건 일치시켜줘야한다. 데이터셋 테스트셋 검증셋
from sklearn.model_selection import train_test_split
t_x,tt_x,t_y,tt_y = train_test_split(X,Y,random_state=42,test_size=0.2)


In [17]:
s_t_x,v_t_x,s_t_y,v_t_y = train_test_split(t_x,t_y,random_state=42,test_size=0.2)

In [5]:
#학습(트레이닝)데이터 테스트데이터 검증데이터
print(s_t_x.shape,tt_x.shape,v_t_x.shape)

(4157, 3) (1300, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(s_t_x,s_t_y)
dt.score(s_t_x,s_t_y), dt.score(v_t_x,v_t_y)

(0.9971133028626413, 0.864423076923077)

In [7]:
from sklearn.model_selection import cross_validate#교차검증
sc=cross_validate(dt,t_x,t_y)
sc

{'fit_time': array([0.00599837, 0.00603104, 0.00500059, 0.00600219, 0.00499964]),
 'score_time': array([0.        , 0.        , 0.00103474, 0.        , 0.00096583]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [8]:
np.mean(sc['test_score'])

0.855300214703487

In [9]:
from sklearn.model_selection import StratifiedKFold#나눠주고 작업하는것
sc1=cross_validate(dt,t_x,t_y,cv=StratifiedKFold())
np.mean(sc1['test_score'])

0.855300214703487

In [10]:
sc_ck=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)#n_splits=10몇개로잘라서 분류할것이냐
sc2=cross_validate(dt,t_x,t_y,cv=sc_ck)
np.mean(sc2['test_score'])

0.8574181117533719

In [20]:
from sklearn.model_selection import GridSearchCV #하이퍼파라미터값에대한 이상적인 값들을 찾아낸다.각데이터를 하나하나 서치
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [25]:
from sklearn.model_selection import GridSearchCV #GridSearchCV
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

In [33]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42),params)
gs.fit(t_x,t_y)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [34]:
dt=gs.best_estimator_
dt.score(t_x,t_y),dt.score(tt_x,tt_y)

(0.9615162593804117, 0.8653846153846154)

In [35]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [36]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [39]:
i=np.argmax(gs.cv_results_['mean_test_score'])

In [40]:
gs.cv_results_['params'][i]

{'min_impurity_decrease': 0.0001}

In [42]:
params={'max_depth':range(5,20,1),
       'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
        'min_samples_split':range(2,100,10)
       }
gs1=GridSearchCV(DecisionTreeClassifier(random_state=42),params)
gs1.fit(t_x,t_y)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [43]:
gs1.best_params_

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [44]:
np.max(gs.cv_results_['mean_test_score'])

0.8681929740134745

랜덤 서치

In [45]:
from scipy.stats import uniform, randint

In [47]:
d=randint(0,10)
d.rvs(5)

array([0, 0, 5, 6, 5])

In [49]:
np.unique(d.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([106,  97,  99,  95, 117, 108,  89,  95,  94, 100], dtype=int64))

In [50]:
d1=uniform(0,1)
d1.rvs(5)

array([0.38301561, 0.8596875 , 0.88379413, 0.527326  , 0.96112518])

#하이퍼파라미터 범위만 설정해주고 랜덤으로 뽑아내는 목적

In [51]:
params={'max_depth':randint(20,50),
       'min_impurity_decrease':uniform(0.0001,0.001),
        'min_samples_split':randint(2,25),
        'min_samples_leaf':randint(1,25)
       }
from sklearn.model_selection import RandomizedSearchCV#랜덤 서칭
rs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,random_state=42)#n_iter 얼마만큼 동작하겠는가
rs.fit(t_x,t_y)


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002651EAB42B0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002651FA13B50>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002651F274790>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002651F2703D0>},
                   random_state=42)

In [52]:
rs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}